In [1]:
import pandas as pd
import numpy as np
import sys, os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 6)

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

PROJECT_PATH = os.getenv('PROJECT_DIR')
sys.path.append(PROJECT_PATH)

In [2]:
from src.data_ingestion.iex_data import IexDataFetcher
from src.get_apis.get_forecast import IexForecast
from src.feature_engineering.build_features import FeatureEngineering
from src.utils import *
from config.paths import *

In [3]:
iex_data = IexDataFetcher()
iex_forecast = IexForecast()
featured_data = FeatureEngineering(PROJECT_PATH)

In [4]:
market_type = 'rtm'

In [5]:
dam_actual = iex_data._get_processed_data('dam')[['datetime', f'mcp_dam']]
rtm_actual = iex_data._get_processed_data('rtm')[['datetime', f'mcp_rtm']]

dam data is already updated up to:  2024-01-18 23:45:00
rtm data is already updated up to:  2024-01-17 23:45:00


In [6]:
dam_rtm_actual = featured_data.merge_dataframes([rtm_actual, dam_actual])
dam_rtm_actual['actual'] = -1  # Initialize with -1 for cases where mcp_dam is equal to mcp_rtm

# Update 'actual' based on conditions
dam_rtm_actual['actual'] = dam_rtm_actual.apply(lambda row: 1 if row['mcp_dam'] > row['mcp_rtm'] else (0 if row['mcp_dam'] < row['mcp_rtm'] else -1), axis=1)

In [7]:
dir_actual = dam_rtm_actual[['datetime', 'actual']]

In [8]:
acc_report = load_pickle(REPORTS_PATH, 'dir_accuracy_report')
acc_start_date = (acc_report['Date'].iloc[-1]\
                + timedelta(days=1)).strftime('%d-%m-%Y')

In [9]:
try:
    sdt = acc_start_date
    tdt = (datetime.now() + timedelta(days=30)).strftime('%d-%m-%Y')
    forecast = iex_forecast._get_processed_forecast(sdt, tdt, market_type)
except:
    print(f'{market_type} accuracy report already updated.')

rtm forecast updated upto:  2024-01-18 23:45:00


In [10]:
df = featured_data.merge_dataframes([forecast, dir_actual])

In [15]:
df['Accuracy'] = (df['actual'] == df['forecast']).astype(int)
df['Date'] = df['datetime'].dt.date

In [16]:
curr_acc = df.groupby(df['Date'])['Accuracy'].sum().reset_index()

In [18]:
acc = pd.concat([acc_report, curr_acc], ignore_index = True)

In [37]:
save_pickle(acc, REPORTS_PATH, 'dir_accuracy_report')

In [ ]:
print(f'Accuracy report for directional generated.')